In [1]:
import numpy as np
import pickle, os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from datetime import datetime
import time
import sys
import os

sys.path.append(os.path.abspath("..")) # Adds the parent folder to sys.path
from utils import *
from model import *
from Neural_Nets import *

In [2]:
relative_path = os.getcwd()
relative_path = os.path.abspath("..")
dataset_fn = relative_path + '/self_supervised' + '/single.p'
prob_features = ['x0', 'xg']

data_file = open(dataset_fn,'rb')
all_data = pickle.load(data_file)[:10000]  # use only part of the dataset for quick testing
data_file.close()
num_train = len(all_data)
print(f"Number of training samples: {num_train}")

X0 = np.vstack([all_data[ii]['x0'].T for ii in range(num_train)])  
XG = np.vstack([all_data[ii]['xg'].T for ii in range(num_train)])  
OBS = np.vstack([all_data[ii]['xg'].T for ii in range(num_train)])  
XX = np.array([all_data[ii]['XX'] for ii in range(num_train)])
UU = np.array([all_data[ii]['UU'] for ii in range(num_train)])
YY = np.concatenate([all_data[ii]['YY'].astype(int) for ii in range(num_train)], axis=1).transpose(1,0,2)
train_data = [{'x0': X0, 'xg': XG}, {'XX': XX, 'UU' : UU}, YY]
Obs_info = np.array([[1.0, 0.0, 0.4, 0.5, 0.0],
        [0.7, -1.1, 0.5, 0.4, 0.0],
        [0.40, -2.50, 0.4, 0.5, 0.0]])


Number of training samples: 10000


# Phase 1: Regression with Feed-forward NNs

In [3]:
# Build the FFNet model
FFNet_reg = Regression(prob_features)
n_features = 6 # the dimension of feature (input vector)
FFNet_reg.setup_data(n_features, train_data, Obs_info)

In [4]:
# FFNet_reg.Y_train.shape
# FFNet_reg.P_train["XX"].shape
# FFNet_reg.n_obs


In [5]:
FFNet_reg.setup_network(depth=4, neurons=1024)
FFNet_reg.model

training_params = {}
training_params['TRAINING_EPOCHS'] = int(1000)
training_params['BATCH_SIZE'] = 200
training_params['CHECKPOINT_AFTER'] = int(1)
training_params['SAVEPOINT_AFTER'] = int(10)
training_params['TEST_BATCH_SIZE'] = 100
training_params['LEARNING_RATE'] = 1e-3
training_params['WEIGHT_DECAY'] = 1e-5
training_params['EARLY_STOPPING_PATIENCE'] = 5

FFNet_reg.train(training_params, verbose=True)

wandb: Currently logged in as: vietanh_penn (vietanh_penn-university-of-pennsylvania) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[Epoch 0], [Iter 46] Saved model at /home/ubuntu22/github/UPenn_Py/SS_Learning_MIQP/self_supervised/regression_20251007_1650.pt
[Epoch 0], [Iter 46] Validation loss: 0.4929 | Validation accuracy (bitwise): 0.9122 | Constraint violation: 0.8474
[Epoch 1], [Iter 91] Validation loss: 0.4912 | Validation accuracy (bitwise): 0.9190 | Constraint violation: 0.8783
[Epoch 2], [Iter 136] Validation loss: 0.4852 | Validation accuracy (bitwise): 0.9298 | Constraint violation: 0.4482
[Epoch 3], [Iter 181] Validation loss: 0.4790 | Validation accuracy (bitwise): 0.9415 | Constraint violation: 0.1802
[Epoch 4], [Iter 226] Validation loss: 0.4769 | Validation accuracy (bitwise): 0.9473 | Constraint violation: 0.1818
[Epoch 5], [Iter 271] Validation loss: 0.4763 | Validation accuracy (bitwise): 0.9492 | Constraint violation: 0.1911
[Epoch 6], [Iter 316] Validation loss: 0.4746 | Validation accuracy (bitwise): 0.9523 | Constraint violation: 0.1654
[Epoch 7], [Iter 361] Validation loss: 0.4753 | Validat

wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


[Epoch 15], [Iter 721] Validation loss: 0.4689 | Validation accuracy (bitwise): 0.9658 | Constraint violation: 0.0617
Early stopping: no improvement for 5 epochs
Final model saved at /home/ubuntu22/github/UPenn_Py/SS_Learning_MIQP/self_supervised/regression_20251007_1650.pt


avg_train_loss,█▄▄▃▂▂▂▂▂▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇████
iteration,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇███
train_loss,██▇▇▇▆▅▄▃▃▃▃▃▂▃▃▃▃▃▃▃▃▃▂▂▂▂▂▁▂▂▂▁▁▁▂▁▂▁▁
val/bitwise_acc,▁▂▃▅▅▆▆▆▇▇██████
val/constraint_violation,██▄▂▂▂▂▂▂▁▁▁▁▁▁▁
val/loss,█▇▆▄▃▃▃▃▂▂▁▁▁▁▁▁
avg_train_loss,0.47014
epoch,15
iteration,720
train_loss,0.4723


# Phase 2: Refine with Self-Supervised Learning

In [ ]:
fn_saved = 'regression_20251007_1650.pt'  
FFNet_reg.setup_network(depth=4, neurons=1024)
FFNet_reg.load_network(fn_saved)
print(FFNet_reg.model)

Loading presaved Hetero GNN classifier model from regression_20251007_1645.pt
FFNet(
  (activation): ReLU()
  (layers): ModuleList(
    (0): Linear(in_features=6, out_features=1024, bias=True)
    (1-3): 3 x Linear(in_features=1024, out_features=1024, bias=True)
    (4): Linear(in_features=1024, out_features=240, bias=True)
  )
)


In [8]:
training_params = {}
training_params['TRAINING_EPOCHS'] = int(1000)
training_params['BATCH_SIZE'] = 200
training_params['CHECKPOINT_AFTER'] = int(1)
training_params['SAVEPOINT_AFTER'] = int(10)
training_params['TEST_BATCH_SIZE'] = 100
training_params['LEARNING_RATE'] = 1e-3
training_params['WEIGHT_DECAY'] = 1e-4
training_params['EARLY_STOPPING_PATIENCE'] = 5

FFNet_reg.SS_train(training_params, verbose=True, penalty_weight=1e0)


[Epoch 0], [Iter 339] Saved model at regression_20251007_1645.pt
[Epoch 0], [Iter 339] Validation loss: 1.2346 | Validation accuracy (bitwise): 0.7742 | Constraint violation: 0.6914


KeyboardInterrupt: 

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x73a337c0f6e0>> (for post_run_cell), with arguments args (<ExecutionResult object at 73a33744c470, execution_count=8 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 73a33744ff20, raw_cell="training_params = {}
training_params['TRAINING_EPO.." transformed_cell="training_params = {}
training_params['TRAINING_EPO.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/ubuntu22/github/UPenn_Py/SS_Learning_MIQP/self_supervised/train_model_SSL.ipynb#X10sZmlsZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe